In [1]:
import os
import re
import math
import random

import datasets
import spacy
import tokenizations
from collections.abc import Mapping

import torch
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence
from functools import partial

from transformers import  DataCollatorForWholeWordMask
from transformers.data.data_collator import tolist, _torch_collate_batch

from transformers import BertConfig, BertTokenizerFast, BertForMaskedLM
from transformers import TrainingArguments, Trainer
from transformers.integrations import WandbCallback, rewrite_logs

In [2]:
pos_tagger = spacy.load('en_core_web_sm')

In [3]:
class BertDataProcessor():
  def __init__(self, hf_dset, hf_tokenizer, max_length, text_col='text', lines_delimiter='\n', minimize_data_size=True, apply_cleaning=True):
    self.hf_tokenizer = hf_tokenizer
    self._current_sentences = []
    self._current_length = 0
    self._max_length = max_length
    self._target_length = max_length

    self.hf_dset = hf_dset
    self.text_col = text_col
    self.lines_delimiter = lines_delimiter
    self.minimize_data_size = minimize_data_size
    self.apply_cleaning = apply_cleaning
    pos_classes = ['ADJ', 'ADP', 'ADV', 'AUX', 'CCONJ', 'DET', 'INTJ', 'NOUN', 'NUM', 'PART', 'PRON', 'PROPN', 'PUNCT', 'SCONJ', 'SYM', 'VERB', 'X']
    self.pos_hash = {c: i for i, c in enumerate(pos_classes)}

  def map(self, **kwargs) -> datasets.arrow_dataset.Dataset:
    num_proc = kwargs.pop('num_proc', os.cpu_count())
    cache_file_name = kwargs.pop('cache_file_name', None)
    if cache_file_name is not None:
        if not cache_file_name.endswith('.arrow'): 
            cache_file_name += '.arrow'        
        if '/' not in cache_file_name: 
            cache_dir = os.path.abspath(os.path.dirname(self.hf_dset.cache_files[0]['filename']))
            cache_file_name = os.path.join(cache_dir, cache_file_name)

    return self.hf_dset.map(
        function=self,
        batched=True,
        cache_file_name=cache_file_name,
        remove_columns=self.hf_dset.column_names,
        disable_nullable=True,
        input_columns=[self.text_col],
        writer_batch_size=10**4,
        num_proc=num_proc,
        **kwargs     
    )

  def __call__(self, texts):
    if self.minimize_data_size: new_example = {'input_ids':[], 'sentA_length':[], 'pos_subword_info':[]}
    else: new_example = {'input_ids':[], 'input_mask': [], 'segment_ids': []}

    for text in texts: # for every doc
      
      for line in re.split(self.lines_delimiter, text): # for every paragraph
        
        if re.fullmatch(r'\s*', line): continue # empty string or string with all space characters
        if self.apply_cleaning and self.filter_out(line): continue
        
        example = self.add_line(line)
        if example:
          for k,v in example.items(): new_example[k].append(v)
      
      if self._current_length != 0:
        example = self._create_example()
        for k,v in example.items(): new_example[k].append(v)

    return new_example

  def filter_out(self, line):
    if len(line) < 80: return True
    return False 

  def clean(self, line):
    # () is remainder after link in it filtered out
    return line.strip().replace("\n", " ").replace("()","")

  def add_line(self, line):
    """Adds a line of text to the current example being built."""
    line = self.clean(line)
    tokens = self.hf_tokenizer.tokenize(line, max_length=512, truncation=True)
    tokids = self.hf_tokenizer.convert_tokens_to_ids(tokens)
    self._current_sentences.append(tokids)
    self._current_length += len(tokids)
    if self._current_length >= self._target_length:
      return self._create_example()
    return None

  def _create_example(self):
    """Creates a pre-training example from the current list of sentences."""
    # small chance to only have one segment as in classification tasks
    if random.random() < 0.1:
      first_segment_target_length = 100000
    else:
      # -3 due to not yet having [CLS]/[SEP] tokens in the input text
      first_segment_target_length = (self._target_length - 3) // 2

    first_segment = []
    second_segment = []
    for sentence in self._current_sentences:
      # the sentence goes to the first segment if (1) the first segment is
      # empty, (2) the sentence doesn't put the first segment over length or
      # (3) 50% of the time when it does put the first segment over length
      if (len(first_segment) == 0 or
          len(first_segment) + len(sentence) < first_segment_target_length or
          (len(second_segment) == 0 and
           len(first_segment) < first_segment_target_length and
           random.random() < 0.5)):
        first_segment += sentence
      else:
        second_segment += sentence

    # trim to max_length while accounting for not-yet-added [CLS]/[SEP] tokens
    first_segment = first_segment[:self._max_length - 2]
    second_segment = second_segment[:max(0, self._max_length -
                                         len(first_segment) - 3)]

    # prepare to start building the next example
    self._current_sentences = []
    self._current_length = 0
    # small chance for random-length instead of max_length-length example
    if random.random() < 0.05:
      self._target_length = random.randint(5, self._max_length)
    else:
      self._target_length = self._max_length

    return self._make_example(first_segment, second_segment)

  def _make_example(self, first_segment, second_segment):
    """Converts two "segments" of text into a tf.train.Example."""
    input_ids = [self.hf_tokenizer.cls_token_id] + first_segment + [self.hf_tokenizer.sep_token_id]

    bert_tokens = self.hf_tokenizer.convert_ids_to_tokens(first_segment)
    sentence = self.hf_tokenizer.decode(first_segment)

    with pos_tagger.select_pipes(enable=['morphologizer', 'tok2vec', 'tagger', 'attribute_ruler']):
      spacy_doc = pos_tagger(sentence)
    spacy_tokens = [t.text for t in spacy_doc]
    pos = torch.tensor([self.pos_hash[t.pos_] for t in spacy_doc])

    # align spacy_tokens to bert_tokens
    a2b, b2a = tokenizations.get_alignments(spacy_tokens, bert_tokens)

    count = 0
    align_index = []
    token_top = -1
    for i in range(len(spacy_tokens)):
      for j in a2b[i]:
        if j > token_top:
          align_index.append(count)
      count += 1
      token_top = a2b[i][-1]
    
    align_index = torch.tensor(align_index)
    # assign pos to bert_tokens
    pos_subword_info = torch.index_select(pos, dim=0, index=align_index)
    pos_subword_info = [-1] + pos_subword_info.tolist() + [-1]

    sentA_length = len(input_ids)
    segment_ids = [0] * sentA_length
    assert len(input_ids) == len(pos_subword_info)

    # if second_segment:
    #   input_ids += second_segment + [self.hf_tokenizer.sep_token_id]
    #   segment_ids += [1] * (len(second_segment) + 1)

    if self.minimize_data_size:
      return {
        'input_ids': input_ids,
        'sentA_length': sentA_length,
        'pos_subword_info': pos_subword_info
      }
    else:
      input_mask = [1] * len(input_ids)
      input_ids += [0] * (self._max_length - len(input_ids))
      input_mask += [0] * (self._max_length - len(input_mask))
      segment_ids += [0] * (self._max_length - len(segment_ids))
      return {
        'input_ids': input_ids,
        'input_mask': input_mask,
        'segment_ids': segment_ids,
      }

In [4]:
hf_tokenizer = BertTokenizerFast.from_pretrained(f"bert-base-uncased")
BertProcessor = partial(BertDataProcessor, hf_tokenizer=hf_tokenizer, max_length=128)

In [8]:
wiki = datasets.load_dataset('wikitext', 'default', cache_dir='./')['train']

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [9]:
wiki[21]

{'text': ' Concept work for Valkyria Chronicles III began after development finished on Valkyria Chronicles II in early 2010 , with full development beginning shortly after this . The director of Valkyria Chronicles II , Takeshi Ozawa , returned to that role for Valkyria Chronicles III . Development work took approximately one year . After the release of Valkyria Chronicles II , the staff took a look at both the popular response for the game and what they wanted to do next for the series . Like its predecessor , Valkyria Chronicles III was developed for PlayStation Portable : this was due to the team wanting to refine the mechanics created for Valkyria Chronicles II , and they had not come up with the " revolutionary " idea that would warrant a new entry for the PlayStation 3 . Speaking in an interview , it was stated that the development team considered Valkyria Chronicles III to be the series \' first true sequel : while Valkyria Chronicles II had required a large amount of trial and

In [11]:
e_wiki = BertProcessor(wiki).map(cache_file_name=f"bert_wikitext_128.arrow", num_proc=4)

Map (num_proc=4):   0%|          | 0/1801350 [00:00<?, ? examples/s]

In [16]:
e_wiki

Dataset({
    features: ['input_ids', 'sentA_length', 'pos_subword_info'],
    num_rows: 762789
})

In [24]:
e_wiki[0]['pos_subword_info']

[-1,
 15,
 15,
 5,
 7,
 7,
 7,
 8,
 12,
 16,
 6,
 6,
 16,
 7,
 12,
 11,
 12,
 4,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 12,
 11,
 12,
 11,
 11,
 11,
 1,
 5,
 7,
 8,
 12,
 12,
 2,
 15,
 1,
 1,
 11,
 11,
 11,
 11,
 11,
 1,
 11,
 12,
 3,
 5,
 0,
 7,
 7,
 12,
 1,
 15,
 7,
 7,
 15,
 1,
 7,
 4,
 7,
 12,
 7,
 1,
 5,
 7,
 0,
 12,
 15,
 1,
 11,
 8,
 1,
 11,
 12,
 10,
 3,
 5,
 0,
 7,
 1,
 5,
 11,
 11,
 11,
 11,
 12,
 15,
 5,
 0,
 7,
 1,
 0,
 4,
 0,
 15,
 12,
 7,
 7,
 7,
 1,
 10,
 7,
 12,
 5,
 7,
 15,
 0,
 1,
 5,
 0,
 7,
 4,
 15,
 5,
 12,
 7,
 7,
 12,
 12,
 5,
 -1]

In [25]:
BertProcessor(wiki).pos_hash

{'ADJ': 0,
 'ADP': 1,
 'ADV': 2,
 'AUX': 3,
 'CCONJ': 4,
 'DET': 5,
 'INTJ': 6,
 'NOUN': 7,
 'NUM': 8,
 'PART': 9,
 'PRON': 10,
 'PROPN': 11,
 'PUNCT': 12,
 'SCONJ': 13,
 'SYM': 14,
 'VERB': 15,
 'X': 16}